## METODO PARA REALIZAR WEB SCRAPPING EN GOOGLE IMAGES ##

1. Descargar y linkar una API de la misma version que mi buscador web (Google Chrome)
2. Realizar una funcion que se dedica a indentificar las imagenes sacando las keywords de nuestras futuras busquedas
3. Procesar otra funcion que se dedique a descargar cada una de las imagenes
4. Organizar las busquedas necesarias para crear mi dataset, y dedicarle un espacio en mi current directory creandoles una carpeta para cada busqueda

In [1]:
from multiprocessing.sharedctypes import Value
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from datetime import datetime as dt
from PIL import Image
import time
import os


#1  Descargar y linkar una API de la misma version que mi buscador web (Google Chrome)

PATH = 'C:/Users/Usuario/Desktop/chromedriver.exe'
wd = webdriver.Chrome(executable_path=PATH)




#2  Realizar una funcion que se dedica a indentificar las imagenes sacando las keywords de nuestras futuras busquedas


def get_images_from_google(wd, delay, max_images, url):
	def scroll_down(wd):
		wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(delay)

	url = url
	wd.get(url)

	image_urls = set()
	skips = 0
	while len(image_urls) + skips < max_images:
		scroll_down(wd)
		thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

		for img in thumbnails[len(image_urls) + skips:max_images]:
			try:
				img.click()
				time.sleep(delay)
			except:
				continue

			images = wd.find_elements(By.CLASS_NAME, "n3VNCb")
			for image in images:
				if image.get_attribute('src') in image_urls:
					max_images += 1
					skips += 1 
					break

				if image.get_attribute('src') and 'http' in image.get_attribute('src'):
					image_urls.add(image.get_attribute('src'))

	return image_urls


#3  Procesar otra funcion que se dedique a descargar cada una de las imagenes


def download_image(down_path, url, file_name, image_type='JPEG',
                   verbose=True):
    try:
        time = dt.now()
        curr_time = time.strftime('%H:%M:%S')
        img_content = requests.get(url).content
        img_file = io.BytesIO(img_content)
        image = Image.open(img_file)
        file_pth = down_path + file_name

        with open(file_pth, 'wb') as file:
            image.save(file, image_type)

        if verbose == True:
            print(f'The image: {file_pth} downloaded successfully at {curr_time}.')
    except Exception as e:
        print(f'Unable to download image from Google Photos due to\n: {str(e)}')




#4  Organizar las busquedas necesarias para crear mi dataset, y dedicarle un espacio en mi current directory creandoles una carpeta para cada busqueda




if __name__ == '__main__':
    google_urls = [
                    "https://www.google.com/search?q=moneda++2+euros+francia+2000&tbm=isch&ved=2ahUKEwiVzqzw4tP7AhVFrxoKHT_yCtAQ2-cCegQIABAA&oq=moneda++2+euros+francia+2000&gs_lcp=CgNpbWcQAzIHCAAQgAQQGDIHCAAQgAQQGDIHCAAQgAQQGDoFCAAQgARQmQZYmQZg7w5oAHAAeACAAbwBiAGOApIBAzEuMZgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=LC6GY5WIEcXear_kq4AN&bih=937&biw=1920&rlz=1C1GCEA_enES1012ES1012",
                   "https://www.google.com/search?q=moneda+altamira+2+euros&rlz=1C1GCEA_enES1012ES1012&sxsrf=ALiCzsZqKL1aprNRcyuV8OLB7e8sycLQwQ:1669883789820&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiC0u7wgdj7AhWI26QKHSHEDgwQ_AUoAXoECAEQAw",
                    "https://www.google.com/search?q=moneda+2+euros+atenas+2004&tbm=isch&ved=2ahUKEwj-tvTxgdj7AhW4hM4BHZe1AEgQ2-cCegQIABAA&oq=moneda+atenas+2+euros&gs_lcp=CgNpbWcQARgAMgYIABAIEB46BggAEAcQHjoICAAQCBAHEB5Q_RBYpBxgsChoAHAAeACAAZoBiAGpB5IBAzIuNpgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=kGeIY_4uuIm6vg-X64LABA&rlz=1C1GCEA_enES1012ES1012",
                    "https://www.google.com/search?q=moneda+2+euros+vaticano+2006&tbm=isch&ved=2ahUKEwjagPmGgtj7AhVEZRoKHT_ABpQQ2-cCegQIABAA&oq=moneda+2+euros+vaticano+2006&gs_lcp=CgNpbWcQAzIHCAAQgAQQGDoECCMQJzoFCAAQgARQkQVY0QlgigtoAHAAeACAAfYBiAHNA5IBBTAuMi4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=vGeIY9qyB8TKab-Am6AJ&rlz=1C1GCEA_enES1012ES1012"]

    nombre_carpeta = ["Moneda Francia1", "Moneda Altamira1","Moneda Atenas1","Moneda Vaticano1"]
    



    coin_path = 'images/SELENIUM 2EURO/'
   
    for lbl in nombre_carpeta:
        if not os.path.exists(coin_path + lbl):
            print(f'Making directory: {str(lbl)}')
            os.makedirs(coin_path+lbl)

    for url_current, lbl in zip(google_urls, nombre_carpeta):
        urls = get_images_from_google(wd, 0, 100, url_current)
    
        for i, url in enumerate(urls):
            download_image(down_path=f'images/SELENIUM 2EURO/{lbl}/', 
                        url=url, 
                        file_name=str(i+1)+ '.jpg',
                        verbose=True) 
    wd.quit()

C:\Users\Usuario\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  from ipykernel import kernelapp as app


Making directory: Moneda Francia1
Making directory: Moneda Altamira1
Making directory: Moneda Atenas1
Making directory: Moneda Vaticano1


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=107.0.5304.108)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C3ACD3+2075859]
	Ordinal0 [0x00BCEE61+1633889]
	Ordinal0 [0x00ACB7BD+571325]
	Ordinal0 [0x00AB2E1A+470554]
	Ordinal0 [0x00B1AA0B+895499]
	Ordinal0 [0x00B2AC96+961686]
	Ordinal0 [0x00B17136+880950]
	Ordinal0 [0x00AEFEFD+720637]
	Ordinal0 [0x00AF0F3F+724799]
	GetHandleVerifier [0x00EEEED2+2769538]
	GetHandleVerifier [0x00EE0D95+2711877]
	GetHandleVerifier [0x00CCA03A+521194]
	GetHandleVerifier [0x00CC8DA0+516432]
	Ordinal0 [0x00BD682C+1665068]
	Ordinal0 [0x00BDB128+1683752]
	Ordinal0 [0x00BDB215+1683989]
	Ordinal0 [0x00BE6484+1729668]
	BaseThreadInitThunk [0x7705FEF9+25]
	RtlGetAppContainerNamedObjectPath [0x77577BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77577B8E+238]
